# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-02-11 18:57 | by soge | Using ris library version: 2.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\Retimed_Corridors

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')


Additional database connection details required:
User name (ris):soge
Password (ris)········


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

### Segments in retimed corridors 19b missing in lion

In [4]:
missing_lion = db.dfquery("""

SELECT DISTINCT rc.segmentid rc_seg, l.segmentid l_seg, rc.rb_layer rcrc, l.rb_layer lrb
FROM working.retimed_corridors rc
LEFT JOIN lion l
ON rc.segmentid = l.segmentid
WHERE l.segmentid is null


""")

In [5]:
print list(missing_lion.rc_seg)

['0152324', '0157955']


### Retimed corridors 19b joined to lion 

In [6]:
joined = db.dfquery("""

SELECT DISTINCT rc.segmentid rc_seg, l.segmentid l_seg
               ,rc.rb_layer rcrc, l.rb_layer lrb
               ,left(rc.shape_leng::varchar,3) rcsl, left(l.shape_length::varchar,3) lsl
FROM working.retimed_corridors rc
LEFT JOIN lion l
ON rc.segmentid = l.segmentid
WHERE l.segmentid is not null

""")

In [7]:
### Differences in shapelength

In [8]:
joined.loc[joined.rcsl!=joined.lsl][['rc_seg','l_seg','rcsl','lsl']]#.l_seg)

,rc_seg,l_seg,rcsl,lsl
91,0016898,0016898,98.,107
92,0016899,0016899,275,268
987,0064252,0064252,157,145
988,0064254,0064254,101,113
1509,0152321,0152321,51.,57.
1510,0152322,0152322,57.,69.
1512,0152325,0152325,27.,16.
1762,0158267,0158267,31.,18.


### Creating Retimed Corridors 19d

In [9]:
db.query("""


DROP TABLE IF EXISTS working.retimed_corridors_19d;
CREATE TABLE working.retimed_corridors_19d AS

WITH data AS(

SELECT DISTINCT l.mft
FROM working.retimed_corridors rc
LEFT JOIN lion l
ON rc.segmentid = l.segmentid
WHERE l.mft is not null

UNION 

SELECT mft FROM lion 
WHERE segmentid::int in (152322,320296,320286,152321
                         ,0000059,0000139)

)

SELECT  * 
FROM lion 
WHERE mft in (SELECT * FROM data)
AND rb_layer in ('G','B');

GRANT ALL ON working.retimed_corridors_19d TO PUBLIC;

""")

- Query run 2021-02-11 18:57:27.610000
 Query time: Query run in 224000 microseconds 
 * Returned 0 rows *


### All Segments that are out of Retimed Corridors Boundary

In [45]:
x=['30749', '61772', '91076', '97022', '97021', '114583', '62876', '62879', '187862', '185058', '30746', '43689', '43688', '187878', '274420', '120160', '92025', '92028', '187878', '84356', '84357', '44528', '44528', '44528', '31346', '31349', '110058', '43245', '87979', '87981', '173406', '173405', '316936', '31367', '30904', '31355', '29000', '118689', '253576', '262754', '274630', '109298', '28999', '274629', '188017', '29002', '29125', '253074', '28990', '28876', '118106', '28992', '28990', '118691', '252750', '109298', '29117', '29115', '28992', '118094', '118691', '28992', '114118', '109164', '274629', '28992', '118693', '118693', '28992', '114119', '299354', '188025', '297839', '29121', '303173', '303172', '118690', '28999', '172198', '252749', '109297', '172199', '29115', '172197', '172200', '43238', '43237', '303172', '187812', '31226', '29124', '188547', '31223', '66005', '65999', '112429', '59350', '170913', '59489', '59350', '170913', '170913', '59350', '158600', '111985', '87836', '87833', '87824', '87823', '44965', '44961', '86542', '114714', '86542', '65873', '65870', '30915', '30916', '30916', '103624', '106008', '42245', '42244', '262411', '296019', '262410', '296018', '262397', '295830', '168327', '28987', '112316', '11826', '112317', '11826', '87813', '87811', '101770', '101770', '101770', '101770', '101770', '87657', '87659', '165248', '165242', '165244', '165243', '165246', '165245', '44670', '274419', '44673', '129787', '129788', '48170', '48304', '43246', '302646', '48305']